<a href="https://colab.research.google.com/github/enjoyPG/2023Gifted/blob/main/students/jwoo428/20230308/%EC%9D%B4%EC%A4%80%EC%9A%B0_%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_5_3%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#트리의 앙상블

#핵심키워드
###앙상블 학습
* 더 좋은 예측 결과를 만들기 위해 여러개의 모델을 훈련하는 머신러닝 알고리즘을 말함
###랜덤포레스트
* 대표적인 결정 트리 기반의 앙상블 학습 방법입니다.
* 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징입니다.
###엑스트라 트리
* 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않는다.
* 대신 랜덤하게 노드를 분할해 과대적합을 감소시킴
###그레이디언트 부스팅
* 랜덤포레스트나 엑스트라 트리와 달리 결정트리를 연속적으로 추가하여 손실함수를 최소화 하는 앙상블 방법임
* 이런 이유로 훈련 속도가 조금 느리지만 더 좋은 성능을 기대할 수 있음
###히스토그램 기반 그레이디언트 부스팅
* 그레이디언트 부스팅의 속도를 개선한 것이다.

#핵심패키지와 함수
###RandomForestClassifier
* 랜덤포레스트 분류클래스
###ExtraTreesClassifier
* 엑스트라 트리 분류 클래스
###GradientBoostingClassifier
* 그레이디언트 부스팅 분류 클래스
###HistGradientBoostingClassifier
* 히스토그램 기반 그레이디언트 부스팅 분류 클래스

###정형데이터와 비정형데이터
* 정형데이터는 어떠한 구조로 되어 있는 데이터이다.(csv나 데이터베이스, 엑셀에 저장하기 쉬움)
* 비정형데이터는 데이터베이스나 엑셀로 표현하기 어려운 것들이다.

##앙상블 학습
* 정형 데이터를 다루는 가장 뛰어난 성과를 내는 알고리즘

##랜덤포레스트
* 랜덤포레스트는 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕에 널리 사용되고 있다.
* 결정트리를 랜덤하게 만들어 결정 트리의 숲을 만들고 각 결정트리의 예측을 통해 최종예측을 만듬.

##부트스트랩 샘플
* 1000개의 샘플이 들어있는 가방에서 100개의 샘플을 뽑느다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣고, 계속해서 100개를 가방에서 뽑으면 중복된 샘플을 뽑을 수 있음. 이러한 샘플을 부트스트랩 샘플이라고 함.

###분류모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택함.
###RandomForestRegressor는 전체특성을 사용한다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input,test_input,train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [6]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


###랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련하기 때문에 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음(이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됨)

###OOB샘플
* 랜덤포레스트는 훈련세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정트리를 훈련함.
* 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플들을 out of bag 즉 oob샘플이라고 한다.

In [7]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


###엑스트라 트리
* 엑스트라 트리는 랜덤포레스트와 비슷하게 작동함
* 엑스트라 트리가 사용하는 결정 트리가 바로 splitter='random'인 결정트리

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et,train_input, train_target,return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


* 엑스트라 트리와 랜덤포레스트의 차이
* 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정트리를 훈련해야 한다. 
* 랜덤하게 노드를 분할하기 때문에 빠른 계산속도가 엑스트라 트리의 장점이다.

In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


###그레이디언트 부스팅
* 그레이디언트 부스팅은 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [11]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,random_state=42)
scores = cross_validate(gb,train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


###히스토그램 기반 그레이디언트 부스팅
* 히스토그램 기반 그레이디언트 부스팅은 정형 데이터를 다루는 머신러닝 알고리즘이다.
* 입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.


In [14]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.8/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


* 히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 permutation_importances()함수를 사용함.
* n_repeats매개변수는 랜덤하게 섞을 횟수를 지정함

In [15]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb,train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [16]:
result = permutation_importance(hgb,test_input, test_target, n_repeats=10,random_state=42, n_jobs=1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [18]:
hgb.score(test_input, test_target)

0.8723076923076923

###사이킷런말고 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여럿있다.
* XGBoost(tree_method매개변수를 'hist'로 지정하면 히스토그램 기반 그레이디언트 부스팅을 사용할 수 있음
* LightGBM(최신기술 다량 함유)

In [20]:
#XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb,train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [22]:
#LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
